# Data Profiler

## Table of Contents:
* [Importing libraries,loading & transforming data](#first-bullet)
* [Using the Existing Labeler](#second-bullet)
* [Training Labeler](#third-bullet)
* [Creating Function to Train Labeler](#fourth-bullet)

### Importing libraries, loading & transforming data <a class="anchor" id="first-bullet"></a>



In [1]:
!pip install dataprofiler

     |████████████████████████████████| 7.5 MB 5.1 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 829 kB 69.7 MB/s 
     |████████████████████████████████| 56 kB 3.1 MB/s 
     |████████████████████████████████| 2.3 MB 23.8 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=78b0c01e7d0eb6aade9f9c7c3d414752623ac7ca4a54da8c0e5c2cf71f7dc385
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This 

In [2]:
import os
import sys
import json
import pandas as pd

try:
    sys.path.insert(0, '..')
    import dataprofiler as dp
except ImportError:
    import dataprofiler as dp

# remove extra tf loggin
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

We will now read in the record linkage dataset

In [3]:
data = dp.Data('/content/drive/MyDrive/Capstone/Client Work/Data/recordlinkage1.csv')
df_data = data.data
df_data.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
0,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
1,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625
2,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168
3,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102
4,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688


### Using the Existing Labeler <a class="anchor" id="second-bullet"></a>


#### We will run the pre-exisitng labeler for structured data for our dataset

In [4]:
labeler = dp.DataLabeler(labeler_type='structured')

# print out the labels and label mapping
print("Labels: {}".format(labeler.labels)) 
print("\n")
print("Label Mapping: {}".format(labeler.label_mapping))
print("\n")

# make predictions and get labels for each cell going row by row
# predict options are model dependent and the default model can show prediction confidences
predictions = labeler.predict(data, predict_options={"show_confidences": True})

# display prediction results
print("Predictions: {}".format(predictions['pred']))
print("\n")

# display confidence results
print("Confidences: {}".format(predictions['conf']))

Labels: ['PAD', 'UNKNOWN', 'ADDRESS', 'BAN', 'CREDIT_CARD', 'DATE', 'TIME', 'DATETIME', 'DRIVERS_LICENSE', 'EMAIL_ADDRESS', 'UUID', 'HASH_OR_KEY', 'IPV4', 'IPV6', 'MAC_ADDRESS', 'PERSON', 'PHONE_NUMBER', 'SSN', 'URL', 'US_STATE', 'INTEGER', 'FLOAT', 'QUANTITY', 'ORDINAL']


Label Mapping: {'PAD': 0, 'UNKNOWN': 1, 'ADDRESS': 2, 'BAN': 3, 'CREDIT_CARD': 4, 'DATE': 5, 'TIME': 6, 'DATETIME': 7, 'DRIVERS_LICENSE': 8, 'EMAIL_ADDRESS': 9, 'UUID': 10, 'HASH_OR_KEY': 11, 'IPV4': 12, 'IPV6': 13, 'MAC_ADDRESS': 14, 'PERSON': 15, 'PHONE_NUMBER': 16, 'SSN': 17, 'URL': 18, 'US_STATE': 19, 'INTEGER': 20, 'FLOAT': 21, 'QUANTITY': 22, 'ORDINAL': 23}


Predictions: ['UNKNOWN' 'UNKNOWN' 'INTEGER' ... 'UNKNOWN' 'DATE' 'DRIVERS_LICENSE']


Confidences: [[0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
# helper functions for printing results

def get_structured_results(results):
    """Helper function to get data labels for each column."""
    columns = []
    predictions = []
    samples = []
    for col in results['data_stats']:
        columns.append(col['column_name'])
        predictions.append(col['data_label'])
        samples.append(col['samples'])

    df_results = pd.DataFrame({'Column': columns, 'Prediction': predictions, 'Sample': samples})
    return df_results

def get_unstructured_results(data, results):
    """Helper function to get data labels for each labeled piece of text."""
    labeled_data = []
    for pred in results['pred'][0]:
        labeled_data.append([data[0][pred[0]:pred[1]], pred[2]])
    label_df = pd.DataFrame(labeled_data, columns=['Text', 'Labels'])
    return label_df
    

pd.set_option('display.width', 100)

In [6]:
# set options to only run the labeler
profile_options = dp.ProfilerOptions()
profile_options.set({"structured_options.text.is_enabled": False, 
                     "int.is_enabled": False, 
                     "float.is_enabled": False, 
                     "order.is_enabled": False, 
                     "category.is_enabled": False, 
                     "chi2_homogeneity.is_enabled": False,
                     "datetime.is_enabled": False,})

profile = dp.Profiler(data, options=profile_options)

results = profile.report()    
print(get_structured_results(results))

INFO:DataProfiler.profilers.profile_builder: Finding the Null values in the columns... 


100%|██████████| 10/10 [00:00<00:00, 163.28it/s]


INFO:DataProfiler.profilers.profile_builder: Calculating the statistics...  (with 4 processes)


100%|██████████| 10/10 [00:00<00:00, 11.55it/s]


          Column       Prediction                                             Sample
0     given_name          UNKNOWN          [pakita, kayla, lachlan, indiana, connor]
1        surname          UNKNOWN        [wasley, boileau, roff, campbell, littlely]
2  street_number          INTEGER                               [67, 303, 9, 14, 16]
3      address_1          UNKNOWN  [hone place, shirlow place, geelong street, ga...
4      address_2          UNKNOWN  [uarah, the lakes retirement village, northwoo...
5         suburb          UNKNOWN  [ryde, balaklava, burwood east, belmont, norwood]
6       postcode          INTEGER                     [5063, 0820, 3943, 3155, 5086]
7          state          UNKNOWN                           [vic, sa, act, qld, vic]
8  date_of_birth             DATE  [19960111, 19240510, 19140203, 19021008, 19280...
9     soc_sec_id  DRIVERS_LICENSE      [3138813, 5659846, 5875656, 4738674, 4578360]


In this example, the results show that the Data Profiler is able to detect integers, URLs, address, and floats appropriately. Unknown is typically strings of text, which is appropriate for those columns.

### Training Labeler <a class="anchor" id="third-bullet"></a>

In [7]:
data.columns

Index(['given_name', 'surname', 'street_number', 'address_1', 'address_2', 'suburb', 'postcode',
       'state', 'date_of_birth', 'soc_sec_id'],
      dtype='object')

#### Here we train the labeler on the record linkage dataset

In [8]:
data = dp.Data('/content/drive/MyDrive/Capstone/Client Work/Data/recordlinkage1.csv')
df = data.data[['given_name', 'surname', 'street_number', 'address_1', 'address_2', 'suburb', 'postcode',
                'state', 'date_of_birth', 'soc_sec_id']]
df.head()

# split data to training and test set
split_ratio = 0.2
df = df.sample(frac=1).reset_index(drop=True)
data_train = df[:int((1 - split_ratio) * len(df))]
data_test = df[int((1 - split_ratio) * len(df)):]

In [10]:
# train a new labeler with column names as labels
if not os.path.exists('data_labeler_saved'):
    os.makedirs('data_labeler_saved')

labeler = dp.train_structured_labeler(
    data=data_train,
    save_dirpath="data_labeler_saved",
    epochs=50,
    default_label="given_name"
)

EPOCH 0, batch_id 1: loss: 3.810373 - acc: 0.272151 - f1_score 0.272151

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2034: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


EPOCH 0 (3s), loss: 2.989107 - acc: 0.387145 - f1_score 0.387145 -- val_f1: 0.065428 - val_precision: 0.079006 - val_recall 0.136769


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2034: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


EPOCH 1 (0s), loss: 1.741082 - acc: 0.520450 - f1_score 0.520450 -- val_f1: 0.088839 - val_precision: 0.285110 - val_recall 0.202605


...

EPOCH 47 (0s), loss: 0.211540 - acc: 0.915519 - f1_score 0.915519 -- val_f1: 0.916550 - val_precision: 0.921293 - val_recall 0.917020


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2034: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


EPOCH 48 (0s), loss: 0.214847 - acc: 0.916644 - f1_score 0.916644 -- val_f1: 0.916960 - val_precision: 0.920340 - val_recall 0.917757


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2034: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


EPOCH 49 (0s), loss: 0.206754 - acc: 0.918166 - f1_score 0.918166 -- val_f1: 0.917930 - val_precision: 0.920696 - val_recall 0.918495


In [11]:
faker_data= dp.Data('/content/drive/MyDrive/Capstone/Client Work/Data/Fake_data.csv')

In [12]:
faker_data.columns

Index(['given_name', 'surname', 'street_number', 'address_1', 'address_2', 'suburb', 'postcode',
       'state', 'date_of_birth', 'soc_sec_id'],
      dtype='object')

#### Here we train the labeler on the dataset that we have created using faker

In [13]:
df1 = faker_data.data[['given_name', 'surname', 'street_number', 'address_1', 'address_2', 'suburb', 'postcode',
                'state', 'date_of_birth', 'soc_sec_id']]
df1.head()

# split data to training and test set
split_ratio = 0.2
df1 = df1.sample(frac=1).reset_index(drop=True)
data_train1 = df1[:int((1 - split_ratio) * len(df1))]
data_test1 = df1[int((1 - split_ratio) * len(df1)):]

In [14]:
# train a new labeler with column names as labels
if not os.path.exists('data_labeler_saved'):
    os.makedirs('data_labeler_saved')

labeler_faker = dp.train_structured_labeler(
    data=data_train1,
    save_dirpath="data_labeler_saved",
    epochs=20,
    default_label="given_name"
)

EPOCH 0, batch_id 0: loss: 4.052550 - acc: 0.145919 - f1_score 0.145919

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2034: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


EPOCH 0 (5s), loss: 1.514228 - acc: 0.616490 - f1_score 0.616490 -- val_f1: 0.362958 - val_precision: 0.428006 - val_recall 0.468975


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2034: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


EPOCH 1 (1s), loss: 0.879483 - acc: 0.717529 - f1_score 0.717529 -- val_f1: 0.427533 - val_precision: 0.473789 - val_recall 0.517597


...

EPOCH 17 (1s), loss: 0.147885 - acc: 0.944804 - f1_score 0.944804 -- val_f1: 0.949420 - val_precision: 0.955999 - val_recall 0.948315


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2034: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


EPOCH 18 (2s), loss: 0.185889 - acc: 0.928039 - f1_score 0.928039 -- val_f1: 0.955145 - val_precision: 0.958481 - val_recall 0.954595


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2034: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


EPOCH 19 (2s), loss: 0.144778 - acc: 0.944255 - f1_score 0.944255 -- val_f1: 0.955584 - val_precision: 0.957835 - val_recall 0.955872


#### Now we will read in the second record linkage dataset. 

#### We will rename all of the columns as well

In [15]:
data1=dp.Data('/content/drive/MyDrive/Capstone/Client Work/Data/recordlinkage2.csv')
# data1= dp.Data('/content/fake.csv')
data1.rename(columns={'given_name': 'col_10', 'surname': 'col_9', 'street_number': 'col_8', 'address_1': 'col_7', 
                      'address_2': 'col_6','suburb': 'col_5', 'postcode': 'col_4', 'state': 'col_3', 'date_of_birth': 
                      'col_2', 'soc_sec_id': 'col_1'}, inplace=True)

In [16]:
data1.head()

,col_10,col_9,col_8,col_7,col_6,col_5,col_4,col_3,col_2,col_1
0,elton,,3,light setreet,pinehill,windermere,3212,vic,19651013,1551941
1,mitchell,maxon,47,edkins street,lochaoair,north ryde,3355,nsw,19390212,8859999
2,,white,72,lambrigg street,kelgoola,broadbeach waters,3159,vic,19620216,9731855
3,elk i,menzies,1,lyster place,,northwood,2585,vic,19980624,4970481
4,,garanggar,,may maxwell crescent,springettst arcade,forest hill,2342,vic,19921016,1366884


#### Using the labeler trained on the record linkage dataset to get predictions for the other record linkage dataset

In [17]:
# predict with the labeler object
profile_options.set({'structured_options.data_labeler.data_labeler_object': labeler})
profile = dp.Profiler(data1, options=profile_options)

# get the prediction from the data profiler
results = profile.report()
print(get_structured_results(results))

INFO:DataProfiler.profilers.profile_builder: Finding the Null values in the columns... 


100%|██████████| 10/10 [00:00<00:00, 139.06it/s]


INFO:DataProfiler.profilers.profile_builder: Calculating the statistics...  (with 4 processes)


100%|██████████| 10/10 [00:00<00:00, 17.95it/s]


   Column     Prediction                                             Sample
0  col_10     given_name          [elijah, noah, cain, samantha, zachariah]
1   col_9        surname          [beatton, ryah, goode, tiahnee, rawlings]
2   col_8  street_number                              [16, 246, 17, 27, 15]
3   col_7      address_1  [gurney kplace, throsselel srreet, marsden str...
4   col_6      address_2  [summer hill, coolsdie, withern house, sundown...
5   col_5         suburb  [macquarie park, glenorf, cantexbury, arana ih...
6   col_4       postcode                     [6149, 3910, 2902, 2090, 2213]
7   col_3          state                          [vic, nsw, nsw, nsw, qld]
8   col_2  date_of_birth  [19391117, 19330313, 19760516, 19890119, 19531...
9   col_1     soc_sec_id      [7533159, 3981694, 3359595, 3938828, 9865461]


#### Using the labeler trained on the dataset generated via faker to get predictions for the other record linkage dataset

In [18]:
# predict with the labeler object
profile_options.set({'structured_options.data_labeler.data_labeler_object': labeler_faker})
profile = dp.Profiler(data1, options=profile_options)

# get the prediction from the data profiler
results = profile.report()
print(get_structured_results(results))

INFO:DataProfiler.profilers.profile_builder: Finding the Null values in the columns... 


100%|██████████| 10/10 [00:00<00:00, 128.55it/s]


INFO:DataProfiler.profilers.profile_builder: Calculating the statistics...  (with 4 processes)


100%|██████████| 10/10 [00:00<00:00, 15.23it/s]


   Column                  Prediction                                             Sample
0  col_10        address_1|given_name                [jesscia, elle, evan, holl, callum]
1   col_9                   address_1         [theodore, walch, ryan, lackher, desantis]
2   col_8               street_number                                [46, 8, 16, 11, 31]
3   col_7                   address_2  [ordell street, holmes c rescent, solandef pla...
4   col_6  suburb|address_2|address_1  [mylandra, kassingbrook, rye park, kiaora, cli...
5   col_5            suburb|address_1  [bittern, magnetic island, thornbury, st peter...
6   col_4               street_number                     [4506, 4350, 2068, 3844, 4306]
7   col_3                   address_1                           [sa, tas, nsw, nsw, vic]
8   col_2               date_of_birth  [19891212, 19280129, 19790719, 19330911, 19941...
9   col_1                  soc_sec_id      [1783041, 9472794, 8822107, 5824638, 7757358]


#### Using the labeler trained on the record linkage dataset to get predictions on the dataset generated via faker

In [19]:
# predict with the labeler object
profile_options.set({'structured_options.data_labeler.data_labeler_object': labeler})
profile = dp.Profiler(faker_data, options=profile_options)

# get the prediction from the data profiler
results = profile.report()
print(get_structured_results(results))

INFO:DataProfiler.profilers.profile_builder: Finding the Null values in the columns... 


/usr/local/lib/python3.7/dist-packages/dataprofiler/profilers/profile_builder.py:2023: UserWarning: The data will be profiled with a sample size of 5000. All statistics will be based on this subsample and not the whole dataset.
  "not the whole dataset.".format(sample_size))
100%|██████████| 10/10 [00:00<00:00, 110.45it/s]


INFO:DataProfiler.profilers.profile_builder: Calculating the statistics...  (with 4 processes)


100%|██████████| 10/10 [00:00<00:00, 16.67it/s]


          Column                Prediction                                             Sample
0     given_name                given_name        [Mark, Melanie, Virginia, Samantha, Nicole]
1        surname                given_name         [Cobb, Calderon, Graham, Contreras, Young]
2  street_number                  postcode                      [240, 29041, 691, 67707, 287]
3      address_1        surname|given_name               [Islands, Oval, Course, Hill, Grove]
4      address_2                 address_2  [Ellis Skyway, Darren Station, Tammy Gardens, ...
5         suburb                 address_2  [Hillview, Brownbury, New Beth, Melissaburgh, ...
6       postcode                  postcode                [77029, 15743, 30853, 60817, 70607]
7          state                 address_2  [New Mexico, New York, New Mexico, Oklahoma, A...
8  date_of_birth             date_of_birth  [19880210, 20141223, 20061028, 19490327, 19440...
9     soc_sec_id  date_of_birth|soc_sec_id  [696239898, 7462

### Creating Function to Train Labeler <a class="anchor" id="fourth-bullet"></a>

In [22]:
def create_labeler(dataframe, n):
  df=dataframe.data[list(dataframe.columns)]
  df.head()
  labeler= dp.train_structured_labeler(
    data=dataframe,
    save_dirpath="data_labeler_saved",
    epochs=n,
    default_label=list(dataframe.columns)[0]
    )
  return labeler

In [23]:
def check_data (dataframe, labeler):
  profile_options.set({'structured_options.data_labeler.data_labeler_object': labeler})
  profile = dp.Profiler(dataframe, options=profile_options)
  # get the prediction from the data profiler
  results = profile.report()
  return(get_structured_results(results))



In [24]:
record_linkage_labeler=create_labeler(data, 50)

EPOCH 0, batch_id 1: loss: 2.962923 - acc: 0.303244 - f1_score 0.303244

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2034: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


EPOCH 0 (3s), loss: 2.212177 - acc: 0.466086 - f1_score 0.466086 -- val_f1: 0.351539 - val_precision: 0.328745 - val_recall 0.504940


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2034: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


EPOCH 1 (0s), loss: 1.464921 - acc: 0.597353 - f1_score 0.597353 -- val_f1: 0.334715 - val_precision: 0.308416 - val_recall 0.485869


...

EPOCH 47 (0s), loss: 0.149212 - acc: 0.945045 - f1_score 0.945045 -- val_f1: 0.931765 - val_precision: 0.938969 - val_recall 0.931815


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2034: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


EPOCH 48 (0s), loss: 0.159448 - acc: 0.942149 - f1_score 0.942149 -- val_f1: 0.931009 - val_precision: 0.940320 - val_recall 0.930857


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2034: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


EPOCH 49 (0s), loss: 0.161740 - acc: 0.939118 - f1_score 0.939118 -- val_f1: 0.931166 - val_precision: 0.939821 - val_recall 0.931328


In [25]:
check_data(data1, record_linkage_labeler)

INFO:DataProfiler.profilers.profile_builder: Finding the Null values in the columns... 


100%|██████████| 10/10 [00:00<00:00, 157.71it/s]


INFO:DataProfiler.profilers.profile_builder: Calculating the statistics...  (with 4 processes)


100%|██████████| 10/10 [00:00<00:00, 17.99it/s]


,Column,Prediction,Sample
0,col_10,given_name,"[renee, gabrielle, julinaa, jakc, jayed]"
1,col_9,surname,"[dunnicjliff, petersen, lowe, lamprey, reinhard]"
2,col_8,street_number,"[42, 108, 23, 8, 136]"
3,col_7,address_1,"[tanj il loop, britten-jones drive, elkedru cl..."
4,col_6,address_2,"[strasus, rockview, top end, tullatoola, roseh..."
5,col_5,suburb,"[aspley, east mitland, airds, upwey, taylors b..."
6,col_4,postcode,"[7009, 4621, 2047, 3058, 6172]"
7,col_3,state,"[nsw, nsw, nsw, nsw, nsw]"
8,col_2,date_of_birth,"[19820129, 19449015, 19151217, 19210202, 19880..."
9,col_1,soc_sec_id,"[7886527, 1148897, 5595371, 1847058, 1378454]"


In [26]:
check_data(faker_data, record_linkage_labeler)

INFO:DataProfiler.profilers.profile_builder: Finding the Null values in the columns... 


/usr/local/lib/python3.7/dist-packages/dataprofiler/profilers/profile_builder.py:2023: UserWarning: The data will be profiled with a sample size of 5000. All statistics will be based on this subsample and not the whole dataset.
  "not the whole dataset.".format(sample_size))
100%|██████████| 10/10 [00:00<00:00, 140.88it/s]


INFO:DataProfiler.profilers.profile_builder: Calculating the statistics...  (with 4 processes)


100%|██████████| 10/10 [00:00<00:00, 18.41it/s]


,Column,Prediction,Sample
0,given_name,given_name,"[Christopher, Rebecca, Erica, Perry, Kimberly]"
1,surname,surname|given_name,"[Wiley, Chen, Wilson, Stewart, Bright]"
2,street_number,postcode,"[92200, 59, 777, 422, 514]"
3,address_1,surname,"[Trafficway, Point, Tunnel, Squares, Circles]"
4,address_2,suburb,"[Hughes Shores, James Mills, Franklin Brook, A..."
5,suburb,suburb,"[Port Erinfurt, East Matthewhaven, New Jamesto..."
6,postcode,postcode,"[25227, 46709, 96057, 98179, 85409]"
7,state,could not determine,"[Mississippi, Alabama, South Dakota, Nevada, O..."
8,date_of_birth,date_of_birth,"[19880213, 19260318, 19690206, 19440905, 19990..."
9,soc_sec_id,soc_sec_id|date_of_birth,"[877701303, 9260063, 65021838, 899010129, 8394..."
